In [1]:
import sqlite3

DB_FILE = "product.db"

def connect_db(db_file=DB_FILE):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    return conn, cursor

def create_table():
    conn, cursor = connect_db()
    try:
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS product (
                Id INTEGER PRIMARY KEY,
                Name TEXT NOT NULL,
                Price REAL NOT NULL,
                Amount INTEGER NOT NULL
            );
        """)
        conn.commit()
        print(f"✅ Đã tạo hoặc đảm bảo bảng 'product' tồn tại trong {DB_FILE}.")
    except sqlite3.Error as e:
        print(f"❌ Lỗi khi tạo bảng: {e}")
    finally:
        conn.close()

# --- CHỨC NĂNG 1: HIỂN THỊ DANH SÁCH SẢN PHẨM ---
def display_products():
    """Hiển thị tất cả các sản phẩm có trong bảng."""
    conn, cursor = connect_db()
    try:
        cursor.execute("SELECT Id, Name, Price, Amount FROM product ORDER BY Id;")
        products = cursor.fetchall()
        
        print("\n--- DANH SÁCH SẢN PHẨM ---")
        if not products:
            print("Hiện tại không có sản phẩm nào trong CSDL.")
            return

        print(f"| {'ID':<4} | {'TÊN SẢN PHẨM':<25} | {'GIÁ':<10} | {'SL':<5} |")
        print("-" * 50)
        for prod in products:
            print(f"| {prod[0]:<4} | {prod[1]:<25} | {prod[2]:<10.2f} | {prod[3]:<5} |")
        print("-" * 50)
        
    except sqlite3.Error as e:
        print(f"❌ Lỗi khi hiển thị sản phẩm: {e}")
    finally:
        conn.close()

# --- CHỨC NĂNG 2: THÊM CÁC SẢN PHẨM ---
def add_product(name, price, amount):
    """Thêm một sản phẩm mới vào bảng product."""
    conn, cursor = connect_db()
    try:
        cursor.execute(
            "INSERT INTO product (Name, Price, Amount) VALUES (?, ?, ?)",
            (name, price, amount)
        )
        conn.commit()
        print(f"\n➕ Đã thêm sản phẩm: '{name}' (ID mới: {cursor.lastrowid}).")
    except sqlite3.IntegrityError as e:
        print(f"❌ Lỗi chèn dữ liệu (Kiểm tra lại Price/Amount không được NULL): {e}")
    except sqlite3.Error as e:
        print(f"❌ Lỗi khi thêm sản phẩm: {e}")
    finally:
        conn.close()

# --- CHỨC NĂNG 3: TÌM KIẾM THÔNG TIN SẢN PHẨM THEO TÊN ---
def search_product_by_name(search_term):
    """Tìm kiếm sản phẩm theo tên (hỗ trợ tìm kiếm gần đúng bằng LIKE)."""
    conn, cursor = connect_db()
    try:
        sql = "SELECT Id, Name, Price, Amount FROM product WHERE Name LIKE ? ORDER BY Name"
        cursor.execute(sql, (f'%{search_term}%',))
        
        results = cursor.fetchall()
        print(f"\n🔍 KẾT QUẢ TÌM KIẾM cho '{search_term}':")
        if not results:
            print("Không tìm thấy sản phẩm nào phù hợp.")
            return

        print(f"| {'ID':<4} | {'TÊN SẢN PHẨM':<25} | {'GIÁ':<10} | {'SL':<5} |")
        print("-" * 50)
        for prod in results:
            print(f"| {prod[0]:<4} | {prod[1]:<25} | {prod[2]:<10.2f} | {prod[3]:<5} |")
        print("-" * 50)

    except sqlite3.Error as e:
        print(f"❌ Lỗi khi tìm kiếm sản phẩm: {e}")
    finally:
        conn.close()

# --- CHỨC NĂNG 4: CẬP NHẬT ĐƠN GIÁ VÀ SỐ LƯỢNG ---
def update_product(prod_id, new_price, new_amount):
    """Cập nhật giá và số lượng của sản phẩm theo ID."""
    conn, cursor = connect_db()
    try:
        cursor.execute(
            "UPDATE product SET Price = ?, Amount = ? WHERE Id = ?",
            (new_price, new_amount, prod_id)
        )
        conn.commit()
        if cursor.rowcount > 0:
            print(f"\n🔄 Đã cập nhật thành công sản phẩm ID={prod_id}. Giá mới: {new_price}, Số lượng mới: {new_amount}")
        else:
            print(f"\n⚠️ Không tìm thấy sản phẩm có ID={prod_id} để cập nhật.")
            
    except sqlite3.Error as e:
        print(f"❌ Lỗi khi cập nhật sản phẩm: {e}")
    finally:
        conn.close()

# --- CHỨC NĂNG 5: XÓA SẢN PHẨM THEO ID ---
def delete_product(prod_id):
    """Xóa sản phẩm theo ID cụ thể."""
    conn, cursor = connect_db()
    try:
        cursor.execute("DELETE FROM product WHERE Id = ?", (prod_id,))
        conn.commit()
        
        if cursor.rowcount > 0:
            print(f"\n🗑️ Đã xóa thành công sản phẩm ID={prod_id}.")
        else:
            print(f"\n⚠️ Không tìm thấy sản phẩm có ID={prod_id} để xóa.")
            
    except sqlite3.Error as e:
        print(f"❌ Lỗi khi xóa sản phẩm: {e}")
    finally:
        conn.close()

# ----------------------------------------------------
# KHỞI CHẠY VÀ DEMO ỨNG DỤNG
# ----------------------------------------------------

def run_demo():
    # 1. Khởi tạo CSDL và Bảng
    create_table()

    # 2. Thêm một số sản phẩm ban đầu
    print("\n--- THÊM DỮ LIỆU BAN ĐẦU ---")
    add_product("Bàn phím cơ", 1500000.0, 10)
    add_product("Chuột không dây", 450000.50, 50)
    add_product("Màn hình LCD 27inch", 6000000.0, 5)
    
    # 3. Hiển thị danh sách sản phẩm
    display_products()
    
    # 4. Tìm kiếm sản phẩm theo tên
    search_product_by_name("Chuột")
    
    # 5. Cập nhật đơn giá và số lượng cho ID=1
    update_product(1, 1200000.0, 15) 
    
    # 6. Hiển thị lại sau khi cập nhật
    display_products()
    
    # 7. Xóa sản phẩm ID=3
    delete_product(3)
    
    # 8. Hiển thị cuối cùng
    display_products()

# Chạy DEMO
run_demo()

✅ Đã tạo hoặc đảm bảo bảng 'product' tồn tại trong product.db.

--- THÊM DỮ LIỆU BAN ĐẦU ---

➕ Đã thêm sản phẩm: 'Bàn phím cơ' (ID mới: 1).

➕ Đã thêm sản phẩm: 'Chuột không dây' (ID mới: 2).

➕ Đã thêm sản phẩm: 'Màn hình LCD 27inch' (ID mới: 3).

--- DANH SÁCH SẢN PHẨM ---
| ID   | TÊN SẢN PHẨM              | GIÁ        | SL    |
--------------------------------------------------
| 1    | Bàn phím cơ               | 1500000.00 | 10    |
| 2    | Chuột không dây           | 450000.50  | 50    |
| 3    | Màn hình LCD 27inch       | 6000000.00 | 5     |
--------------------------------------------------

🔍 KẾT QUẢ TÌM KIẾM cho 'Chuột':
| ID   | TÊN SẢN PHẨM              | GIÁ        | SL    |
--------------------------------------------------
| 2    | Chuột không dây           | 450000.50  | 50    |
--------------------------------------------------

🔄 Đã cập nhật thành công sản phẩm ID=1. Giá mới: 1200000.0, Số lượng mới: 15

--- DANH SÁCH SẢN PHẨM ---
| ID   | TÊN SẢN PHẨM           